In [23]:
import os
import numpy as np
import matplotlib.pyplot as plt
import random
import tensorflow as tf

from keras.models import Model
from keras import backend as K
from keras import Input
from keras import Sequential
from keras.layers import Dense, BatchNormalization, Conv3DTranspose, LeakyReLU, Reshape, ReLU, Conv3D, Flatten, MaxPooling3D

from tqdm import tqdm

%matplotlib inline

In [24]:
def show3D(builds, binary=True, textArr=None):
    fig = plt.figure(figsize=(20,7))
    col = 8
    
    if binary:
        colors = 'red'
    else:
        colors = 'red' #replace later with colors for each block id value [0-12]
    
    for i, build in enumerate(builds):
        ax = fig.add_subplot(int(len(builds) / col) + 1, col, i + 1,  projection='3d')
        ax.voxels(build, edgecolor="k", facecolors=colors, linewidth=0.5)
        plt.axis('off')
        if textArr != None:
            plt.title(textArr[i])
    plt.show()

In [25]:
def one_hot(m,channels):
  map4d = np.zeros((m.shape[0],m.shape[1],m.shape[2],len(channels)))
  for r in range(m.shape[0]):
    for c in range(m.shape[1]):
      for z in range(m.shape[2]):
        v = list(channels).index(m[r][c][z])
        map4d[r][c][z][v] = 1
  return map4d

In [26]:
def cropHouse(h):
    # argwhere will give you the coordinates of every non-zero point
    true_points = np.argwhere(h)
    # take the smallest points and use them as the top left of your crop
    top_left = true_points.min(axis=0)
    # take the largest points and use them as the bottom right of your crop
    bottom_right = true_points.max(axis=0)
    out = h[top_left[0]:bottom_right[0]+1,  # plus 1 because slice isn't
              top_left[1]:bottom_right[1]+1,top_left[2]:bottom_right[2]+1]  # inclusive
    
    return out

def houseTrans(h,s=(16,16,16)):
    hts = []
    s2 = cropHouse(h)
    s2s = s2.shape
    ds = (s[0]-s2s[0],s[1]-s2s[1],s[2]-s2s[2])
    RATE = 2
    #print(ds)
    #for x in range(1):
    for x in range(0,s[0]-s2s[0]+1,RATE):
        #for y in range(1):
        for y in range(0,s[1]-s2s[1],RATE):
            for z in range(1):
            #for z in range(s[2]-s2s[2]+1):
                thouse = np.zeros(shape=s)
                thouse[x:x+s2s[0],y:y+s2s[1],z:z+s2s[2]] = s2.copy()
                hts.append(thouse)
    return hts

In [27]:
compression_list = [[0, 6, 26, 27, 28, 30, 55, 63, 65, 66, 68, 69, 70, 72, 77, 97, 104, 117, 127, 131, 132, 143, 144, 147, 148, 149, 167, 171, 50, 51, 76, 105, 123, 64, 71, 193, 194, 195, 196, 197, 8, 9, 10, 11, 213],[1, 4, 7, 29, 33, 34, 46, 48, 49, 52, 54, 61, 87, 89, 98, 45, 112, 120, 121, 139, 155, 158, 168, 169, 201, 202, 206, 215, 216, 218, 219, 220, 221, 222, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 35, 14, 15, 16, 21, 56, 73, 129, 153],[2, 3, 82, 88, 159, 172, 173, 214, 12, 13, 19, 24, 179, 78, 79, 80, 174],[5, 17, 25, 47, 58, 84, 96, 116, 130, 140, 146, 151, 154, 162],[18, 31, 32, 81, 86, 91, 103, 106, 161, 170, 199, 200, 207],[20, 92, 102, 160, 95],[22, 23, 41, 42, 57, 118, 133, 138, 145, 152, 165],[37, 38, 39, 40, 59, 83, 110, 115, 141, 142, 175],[43, 44, 92, 125, 126, 181, 182, 204, 205],[53, 67, 108, 109, 114, 128, 134, 135, 136, 156, 163, 164, 180, 203],[85, 101, 107, 113, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 198]]
real_comp_list = [0,1,2,5,12,18,20,42,53,85,126]
def getCompLoc(bi):
  for i in range(len(compression_list)):
    if int(bi) in compression_list[i]:
      return i
  return 0

#remove oob values
def compress_house(house):
    poss_values = sum(compression_list,[])
    alter_house = abs(house)
    for i in range(alter_house.shape[0]):
        for j in range(alter_house.shape[1]):
            for k in range(alter_house.shape[2]):
                  alter_house[i][j][k] = getCompLoc(alter_house[i][j][k])
    return alter_house

# reassign the block id to the real compressed version
def reassignBlock(house):
    alter_house = np.copy(house)
    for i in range(alter_house.shape[0]):
        for j in range(alter_house.shape[1]):
            for k in range(alter_house.shape[2]):
                alter_house[i][j][k] = real_comp_list[int(alter_house[i][j][k])]
    return alter_house

In [49]:
#import training data
HOUSES_PAINTED = []
HOUSES_BINARY = []
HOUSES_NORMAL = []

house_combined = np.load('combined.npy')
blocks = []
with tqdm(total=len(house_combined)) as pbar:
    #id_set = range(len(compression_list))
    # id_set = sum(compression_list,[])
    id_set = range(len(real_comp_list))
    for h in house_combined:
        
        # houses look rotated... just rotate them back
        h = np.rot90(h,axes=(0,2))
        
        # remove bottom layer (got the ground as well) - i can't believe i got it right on the first try...
        h = h[3:, 3:, 1:-2]

        #compress the house to the block compression set
        h = compress_house(h)

        #rotated
        rot_set = []
        rot_set.append(h)
        rot_set.append(np.rot90(h,axes=(0,1)))
        rot_set.append(np.rot90(h,axes=(1,0)))
        rot_set.append(np.rot90(np.rot90(h,axes=(1,0)),axes=(1,0)))

        #translate houses
        for hr in rot_set:
          tds = houseTrans(hr,(16,16,16))
          tdi = list(range(len(tds)))
          random.shuffle(tdi)

          for hti in tdi[:int(len(tdi)/6)]:
            ht = tds[hti]
            #one hot encode
            he = one_hot(np.array(ht),id_set)
            HOUSES_PAINTED.append(he)

            #make binary
            idx = np.nonzero(ht)
            hb = np.zeros(shape=ht.shape)
            for i in range(len(idx[0])):
                a,b,c = idx
                hb[a[i]][b[i]][c[i]] = 1

            HOUSES_BINARY.append(np.expand_dims(hb,axis=-1))

            #save the original
            HOUSES_NORMAL.append(ht)

        pbar.update(1)

HOUSES_PAINTED = np.array(HOUSES_PAINTED)
HOUSES_BINARY = np.array(HOUSES_BINARY)
HOUSES_NORMAL = np.array(HOUSES_NORMAL)

100%|██████████| 144/144 [00:11<00:00, 12.72it/s]


In [50]:
print(HOUSES_BINARY.shape)
print(HOUSES_PAINTED.shape)

(1648, 16, 16, 16, 1)
(1648, 16, 16, 16, 11)


In [51]:
hehe = random.choice(range(len(HOUSES_NORMAL)))
print(f"#{hehe}")
print(np.unique(HOUSES_NORMAL))
print(np.unique(HOUSES_NORMAL[hehe]))
print(np.unique(compress_house(HOUSES_NORMAL[hehe])))
print(np.unique(np.argmax(HOUSES_PAINTED[hehe],axis=-1)))

#146
[ 0.  1.  2.  3.  4.  5.  7.  8.  9. 10.]
[0. 1. 2. 3. 5.]
[0. 1. 2. 3.]
[0 1 2 3 5]


In [52]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Conv2D
from keras import backend as K

#fuck it, try a thrown together neural network (take in as input a binary house and output a painted house)
class Painter(Model):
    def __init__(self,shape,batches=16,channels=81):
        super(Painter, self).__init__()
        self.batches = batches
        self.shape = shape
        self.channels = channels
        self.model = Sequential([
          Input(shape=self.shape),
          # Conv2D(1024,2,activation="relu", padding="same"),
          Conv2D(512,2,activation="relu", padding="same"),
          Conv2D(256,2,activation="relu", padding="same"),
          Conv2D(128,2,activation="sigmoid", padding="same"),
          # Conv2D(81,2,activation="sigmoid", padding="same"),
          Dense(self.channels,activation="softmax")
        ])
    def exportMod(self,label):
        self.model.save(f"painter-{label}.h5")
        
    def importMod(self,label):
        self.model = keras.models.load_model(f"painter-{label}.h5")


def mask_loss(y_true,y_pred):
    #cast values
    zero = tf.constant(0, dtype=tf.float32)
    y_true2 = tf.cast(y_true,tf.float32)
    y_pred2 = tf.cast(y_pred,tf.float32)
    
    #apply mask
    # mask = tf.cast(tf.where(tf.not_equal(y_true2, zero),1,0),tf.float32)
    # mask_pred = tf.math.multiply(y_pred2,mask)

    # return tf.losses.mean_squared_error(y_true2,mask_pred)


    #eval non-zeros
    # y_true2 = y_true2 * tf.cast((y_true2 != 0), 'float32')
    # y_pred2 = y_pred2 * tf.cast((y_true2 != 0), 'float32')

    # error = K.sqrt(K.mean(K.square(y_pred2 - y_true2), axis=-1)) 
    # return error

    return K.categorical_crossentropy(y_true2, y_pred2)

In [53]:
EPOCHS = 20
BATCHES = 16
CHANNELS = HOUSES_PAINTED.shape[-1]

painter_model = Painter((16,16,16,1),BATCHES,CHANNELS)
painter_model.model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),loss=mask_loss)
painter_model.model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 16, 16, 16, 512)   2560      
                                                                 
 conv2d_7 (Conv2D)           (None, 16, 16, 16, 256)   524544    
                                                                 
 conv2d_8 (Conv2D)           (None, 16, 16, 16, 128)   131200    
                                                                 
 dense_2 (Dense)             (None, 16, 16, 16, 11)    1419      
                                                                 
Total params: 659,723
Trainable params: 659,723
Non-trainable params: 0
_________________________________________________________________


In [54]:
#test the loss function
# a = np.expand_dims(y_dat[0],axis=0)
# b = painter_model.model.predict(np.expand_dims(X_dat[0],axis=0))
# mask_loss(a,b)

In [55]:
X_dat = HOUSES_BINARY
y_dat = HOUSES_PAINTED

with tf.device('/device:GPU:0'):
  painter_model.model.fit(X_dat,y_dat,epochs=EPOCHS,shuffle=True)

Epoch 1/20
52/52 [==============================] - 12s 214ms/step - loss: 0.6562
Epoch 2/20
52/52 [==============================] - 11s 220ms/step - loss: 0.1566
Epoch 3/20
52/52 [==============================] - 12s 222ms/step - loss: 0.0845
Epoch 4/20
52/52 [==============================] - 12s 223ms/step - loss: 0.0647
Epoch 5/20
52/52 [==============================] - 11s 221ms/step - loss: 0.0619
Epoch 6/20
52/52 [==============================] - 11s 220ms/step - loss: 0.0604
Epoch 7/20
52/52 [==============================] - 11s 218ms/step - loss: 0.0595
Epoch 8/20
52/52 [==============================] - 11s 219ms/step - loss: 0.0588
Epoch 9/20
52/52 [==============================] - 11s 219ms/step - loss: 0.0580
Epoch 10/20
52/52 [==============================] - 11s 220ms/step - loss: 0.0574
Epoch 11/20
52/52 [==============================] - 11s 221ms/step - loss: 0.0570
Epoch 12/20
52/52 [==============================] - 11s 221ms/step - loss: 0.0566
Epoch 13/20
5

In [56]:
#test output
id_set = range(len(real_comp_list))
hxi = random.choice(range(len(X_dat)))
hx = X_dat[hxi]
p = painter_model.model.predict(np.expand_dims(hx,axis=0))
poh = one_hot(np.argmax(p,axis=-1).squeeze(),id_set)
print(y_dat[hxi].shape)
print(poh.shape)
# print(np.unique(y_dat[hxi]))
# print(np.unique(poh))
print(np.unique(np.argmax(y_dat[hxi],axis=-1).squeeze()))
print(np.unique(np.argmax(p,axis=-1).squeeze()))

(16, 16, 16, 11)
(16, 16, 16, 11)
[0 1 2 3 8]
[0 1 2 3 8]


In [57]:
painter_model.exportMod(f"{EPOCHS}ep-ALT")